# A simple encryption scheme available in ResNet

Users of the [ResNet app](https://resnet.me) have the option to use a simple but effective encryption scheme to send messages, i.e., a subject of maximum length of 200 characters and a message text of maximum length of 3000 characters.

The encryption scheme applies the Blum-Blum-Shub algorithm [1] to generate a pseudo-random sequence $x, x^2, x^4, x^8, \ldots\pmod{n}$, where $x$ and $n$ are based on a nonce (a number only used once) and a shared secret (a password transmitted over a secure channel, e.g., at a face-to-face meeting), and that is used as a binary one-time pad XOR-ed (the ^ operator in Python) with the subject and the message text. This notebook is a Python translation of the used Kotlin code for Android devices and Swift code for iOS.

To demonstrate the strength of this encryption scheme - and also for fun 😊 - we created a PHP version of the encryption scheme available [here](https://resnet.me/bbs/BBS.php). The message text is the private key of the Solana address [CSfT8di8n1mP7KHQaHXniEGdKsWaqkyifsVVAokFbw3x](https://solscan.io/account/CSfT8di8n1mP7KHQaHXniEGdKsWaqkyifsVVAokFbw3x) and we therefore offer 133 USD for cracking this encryption scheme.

[1] Blum, L.; Blum, M.; Shub, M. (1986). "A Simple Unpredictable Pseudo-Random Number Generator". SIAM Journal on Computing. 15 (2). Society for Industrial & Applied Mathematics (SIAM): 364–383, doi:10.1137/0215025

In [1]:
import hashlib
from sympy import nextprime, isprime
from math import gcd

if "nonce" not in globals():
    nonce = 0


def bbs(seed, n, count):
    """Generates a pseudo-random sequence using the Blum-Blum-Shub algorithm."""
    result = []
    x = seed
    for _ in range(count):
        x = (x * x) % n
        result.append(x % 256)
    return result


def encrypt(subject: str, message: str, shared_secret: str) -> tuple[str, str]:
    global nonce
    # Increment nonce
    nonce += 1

    # Create bytes for hashing
    bytes_to_hash = (str(nonce) + shared_secret).encode()
    digest = hashlib.sha512(bytes_to_hash).digest()

    # Split digest into parts
    d1 = bytearray(digest[0:5])
    d2 = bytearray(digest[5:10])
    d3 = bytearray(digest[10:15])
    d4 = bytearray(digest[15:20])
    d5 = bytearray(digest[24:44])
    d6 = bytearray(digest[44:64])

    # Modify first byte of each array
    for d in [d1, d2, d3, d4, d5, d6]:
        d[0] |= 128

    # Generate primes
    p1 = nextprime(int.from_bytes(d1, byteorder="big"))
    p2 = nextprime(int.from_bytes(d2, byteorder="big"))
    q1 = nextprime(int.from_bytes(d3, byteorder="big"))
    q2 = nextprime(int.from_bytes(d4, byteorder="big"))

    while not isprime(2 * p1 * p2 + 1):
        p2 = nextprime(p2)

    while not isprime(2 * q1 * q2 + 1):
        q2 = nextprime(q2)

    # Calculate modulus
    n = (2 * p1 * p2 + 1) * (2 * q1 * q2 + 1)

    # Convert subject and message to bytes
    subject_bytes = subject.encode("utf-8")
    message_bytes = message.encode("utf-8")

    # Generate pads
    pad_subject = bbs(int.from_bytes(d5, byteorder="big"), n, len(subject_bytes))
    pad_message = bbs(int.from_bytes(d6, byteorder="big"), n, len(message_bytes))

    # Encrypt subject
    encrypted_subject = f"{nonce} "
    encrypted_message = ""
    chars = "0123456789abcdef"

    for j in range(len(subject_bytes)):
        idx = subject_bytes[j] ^ pad_subject[j]
        encrypted_subject += chars[idx & 15]
        encrypted_subject += chars[(idx & 240) >> 4]

    # Encrypt message
    for j in range(len(message_bytes)):
        idx = message_bytes[j] ^ pad_message[j]
        encrypted_message += chars[idx & 15]
        encrypted_message += chars[(idx & 240) >> 4]

    return encrypted_subject, encrypted_message


def decrypt(
    encrypted_subject: str, encrypted_message: str, shared_secret: str
) -> tuple[str, str]:
    # Split encrypted subject into components
    parts = encrypted_subject.split(" ", 2)
    nonce = parts[0]
    enc_subject = parts[1]

    bytes_to_hash = (nonce + shared_secret).encode()
    digest = hashlib.sha512(bytes_to_hash).digest()

    # Split digest into parts
    d1 = bytearray(digest[0:5])
    d2 = bytearray(digest[5:10])
    d3 = bytearray(digest[10:15])
    d4 = bytearray(digest[15:20])
    d5 = bytearray(digest[24:44])
    d6 = bytearray(digest[44:64])

    # Modify first byte of each array
    for d in [d1, d2, d3, d4, d5, d6]:
        d[0] |= 128

    # Generate primes
    p1 = nextprime(int.from_bytes(d1, byteorder="big"))
    p2 = nextprime(int.from_bytes(d2, byteorder="big"))
    q1 = nextprime(int.from_bytes(d3, byteorder="big"))
    q2 = nextprime(int.from_bytes(d4, byteorder="big"))

    while not isprime(2 * p1 * p2 + 1):
        p2 = nextprime(p2)

    while not isprime(2 * q1 * q2 + 1):
        q2 = nextprime(q2)

    # Calculate modulus
    n = (2 * p1 * p2 + 1) * (2 * q1 * q2 + 1)

    # Initialize arrays for decryption
    chars = "0123456789abcdef"
    subject_bytes = bytearray(len(enc_subject) // 2)
    message_bytes = bytearray(len(encrypted_message) // 2)

    # Generate pads
    pad_subject = bbs(int.from_bytes(d5, byteorder="big"), n, len(subject_bytes))
    pad_message = bbs(int.from_bytes(d6, byteorder="big"), n, len(message_bytes))

    # Decrypt subject
    for j in range(len(subject_bytes)):
        t = chars.index(enc_subject[2 * j]) + (chars.index(enc_subject[2 * j + 1]) << 4)
        subject_bytes[j] = t ^ pad_subject[j]

    # Decrypt message
    for j in range(len(message_bytes)):
        t = chars.index(encrypted_message[2 * j]) + (
            chars.index(encrypted_message[2 * j + 1]) << 4
        )
        message_bytes[j] = t ^ pad_message[j]

    # Convert byte arrays back to strings
    subject = subject_bytes.decode("utf-8")
    message = message_bytes.decode("utf-8")

    return subject, message

In [2]:
my_subject = "Subject 🌱"
my_message = "Message text... ✉️"
my_password = "Password123"
cipher = encrypt(my_subject, my_message, my_password)
plain = decrypt(cipher[0], cipher[1], my_password)
print(cipher)
print(plain)

('1 59f9a9514c2ca9172d236fd4', 'e673225f6ed204b22ae2b9b954a57f3d0bdfcc642d21')
('Subject 🌱', 'Message text... ✉️')
